## **Mapeamento Galp**

##### **___Informações___**

**Projeto:** Galp

**Autor:** Lucas Pimenta dos Santos Monteiro

**Data:** 24/02/2025

##### **___Objetivo___**

A ideia desse cógido é automatizar a visualização de uma imagem .svg gerada no Inkscape de acordo com o output do programa (report.csv) e os dados de entrada definidos (entry_data.csv).

Para isso foram criados rótulos no Inkscape que identificam cada elemento de texto a partir do inkscape:label. Cada label recebeu o nome equivalente a variável proveniente do report.csv e entry_data.csv que representa o valor desejado.

##### **Importando as Bibliotecas**

In [228]:
import pandas as pd
import xml.etree.ElementTree as ET

In [229]:
entry_data = pd.read_csv("entry_data.csv")
report = pd.read_csv("report.csv")

##### **Verificando os Dados**

In [230]:
entry_data.head()

,Entry data:,Unnamed: 1,Unnamed: 2
0,Entry feed mass flow rate =,600.0000000000,kg/h
1,Vacuum pressure =,-495.8300000000,mbar
2,Membrane area =,12.9600000000,m²
3,Membrane thickness =,100.0000000000,microns
4,Membrane porosity =,85.0000000000%,NaN


In [231]:
report.head()

,Time (s),Desal - Coolant mass flow rate (kg/h),Desal - Hot feedwater inlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Cold feedwater inlet temperature (°C),Desal - Cold feedwater outlet temperature (°C),Desal - Feed membrane temperature (°C),Desal - Membrane gap temperature (°C),Desal - Gap film boundary temperature (°C),Desal - Film wall temperature (°C),...,Motor - Coolant efficiency,Motor - Energy input (kW),Motor - Electrical energy generated/demanded (kWh),Motor - Fuel consumed (L),Mix Valve - Mass flow rate in from reservoir(kg/s),Mix Valve - Temperature in from reservoir(kg/s),Mix Valve - Mass flow rate in from motor(kg/s),Mix Valve - Temperature in from motor(kg/s),Mix Valve - Out Mass flow rate (kg/s),Mix Valve - Out Temperature (°C)
0,126.0,600.0,60.439246,35.375980,30.173678,55.235461,47.405935,46.205715,44.923790,43.538544,...,23.235384,60.272723,0.735,0.219162,0.5,53.726922,0.332177,70.627097,0.832177,60.472905
1,252.0,600.0,61.608506,35.514630,30.157108,56.249629,48.042971,46.817993,45.509037,44.068360,...,23.235384,60.272723,1.470,0.438324,0.5,54.967065,0.366624,70.744518,0.866624,61.641688
2,378.0,600.0,62.721004,35.645009,30.142841,57.217631,48.648495,47.401259,46.067971,44.574576,...,23.235384,60.272723,2.205,0.657486,0.5,56.153735,0.406112,70.879154,0.906112,62.753550
3,504.0,600.0,63.779898,35.767728,30.130567,58.141698,49.224315,47.957037,46.601811,45.058258,...,23.235384,60.272723,2.940,0.876649,0.5,57.290017,0.451497,71.033926,0.951497,63.811673
4,630.0,600.0,64.788309,35.883366,30.120016,59.024102,49.772209,48.486851,47.111812,45.520513,...,23.235384,60.272723,3.675,1.095811,0.5,58.378785,0.503720,71.212060,1.003720,64.819205


##### **Definindo as Colunas que serão utilizadas de cada dataframe**

In [232]:
userows_entry_data = [
    "Cooling heat exchanger flow rate =",
    "Brine flow rate out of the regulating tank =",
    "Brackish water temperature at the entrance of the regulating tank =",
    "Brackish water salinity at the entrance of the regulating tank =",
    "Temperature at the entrance of the cold side =",
    "Heating heat exchanger mass flow rate =",
    "Motor power demanded/generated ="
]

usecols_report = [
"Time (s)",
"Desal - Total water produced (kg)",
"Desal - Distilled water production rate (kg/h)",
"FeedTank - Brackish water inlet flow rate (kg/h)",
"ColdHex - Heat transfer rate (W)",
"ColdHex - Cold side outlet temperature (°C)",
"Desal - Hot feedwater outlet temperature (°C)",
"Desal - Hot feedwater outlet mass flow rate (kg/h)",
"Desal - Hot feedwater outlet salinity (wt%)",
"HotHex - Cold side salinity (wt%)",
"ColdHex - Hot side outlet temperature (°C)",
"ColdHex - Hot side salinity (wt%)",
"HotHex - Cold side salinity (wt%)",
"HotHex - Heat transfer rate (W)",
"HotHex - Hot side outlet temperature (°C)",
"HotHex - Hot side inlet temperature (°C)",
"HotHex - Cold side outlet temperature (°C)",
"ColdHex - Hot side inlet temperature (°C)",
"ColdHex - Hot side inlet mass flow rate (kg/h)",
"HotHex - Cold side inlet temperature (°C)",
"Time (s)",
"SolarField - Solar irradiance (W/m²)", # segundo 11088 -> tempo de máximo
"SolarField - Efficiency",
"SolarField - Solar field inlet temperature (°C)",
"SolarField - Solar field outlet temperature (°C)",
"CircuitFromSolar - Outlet temperature (°C)",
"CircuitToSolar - Inlet temperature (°C)",
"ColdHex - Hot side salinity (wt%)",
"FeedTank - Water salinity (wt%)",
"FeedTank - Water temperature (°C)",
"Desal - Film wall temperature (°C)",
"Desal - Gap film boundary temperature (°C)",
"HCPV - Inlet temperature (°C)",
"HCPV - Outlet temperature (°C)",
"HCPV - Mass flow rate (kg/s)",
"HCPV - Heat transfer rate recovered from the solar cells (kW)",
"HCPV - Electrical power (kW)",
"HCPV - Global panel electrical efficiency (%)",
"HCPV - Electrical energy generated (kWh)",
"CircuitToSolar - Mass flow rate (kg/s)",
"ETC - Efficiency",
"ETC - ETC inlet temperature (°C)",
"ETC - ETC outlet temperature (°C)",
"ETC - Mass flow rate (kg/s)",
"PVT - PVT inlet temperature (°C)",
"PVT - Thermal Efficiency",
"PVT - Electrical power output (W)",
"PVT - Electrical Efficiency",
"PVT - Electrical energy generated (kWh)",
"PVT - PVT outlet temperature (°C)",
"PVT - Mass flow rate (kg/s)",
"PVT - Heat transfer rate (W)",
"ETC - Heat transfer rate (W)",
"SolarField - Heat transfer rate (W)",
"Reservoir - Temperature (°C)",
"Mix Valve - Mass flow rate in from reservoir(kg/s)",
"Mix Valve - Temperature in from reservoir(kg/s)",
"Mix Valve - Out Temperature (°C)",
"Mix Valve - Out Mass flow rate (kg/s)",
"Motor - Outlet mass flow rate (kg/s)",
"Motor - Outlet temperature motor (°C)",
"Motor - Radiator in mass flow rate (kg/s)",
"Motor - Radiator bypass mass flow rate (kg/s)",
"Motor - Recirculation mass flow rate (kg/s)",
"Motor - Nominal Mass flow rate (kg/s)",
"Motor - Mixture temperature engine (°C)",
"Motor - Mixture temperature radiator (°C)",
"Motor - Oulet temperature radiator (°C)",
"Motor - Heat coolant (kW)",
"Motor - System efficiency",
"Motor - Electrical energy generated/demanded (kWh)",
"Motor - Fuel consumed (L)",
"Motor - Fuel consumption (L/h)",
"Motor - Fuel consumption (g/kWh)"
]

##### Filtrando os Dataframes de Acordo com as Colunas

In [233]:
entry_data_filtrado = entry_data[entry_data['Entry data:'].isin(userows_entry_data)]
entry_data_filtrado.head()

,Entry data:,Unnamed: 1,Unnamed: 2
19,Cooling heat exchanger flow rate =,2000.0000000000,kg/h
20,Temperature at the entrance of the cold side =,30.0000000000,°C
23,Brackish water salinity at the entrance of the...,1.0000000000,wt%
24,Brackish water temperature at the entrance of ...,25.0000000000,°C
25,Brine flow rate out of the regulating tank =,0.0000000000,kg/h


In [234]:
report_filtrado = report.filter(items=usecols_report)
report_filtrado.head()

,Time (s),Desal - Total water produced (kg),Desal - Distilled water production rate (kg/h),FeedTank - Brackish water inlet flow rate (kg/h),ColdHex - Heat transfer rate (W),ColdHex - Cold side outlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Hot feedwater outlet mass flow rate (kg/h),Desal - Hot feedwater outlet salinity (wt%),HotHex - Cold side salinity (wt%),...,Motor - Nominal Mass flow rate (kg/s),Motor - Mixture temperature engine (°C),Motor - Mixture temperature radiator (°C),Motor - Oulet temperature radiator (°C),Motor - Heat coolant (kW),Motor - System efficiency,Motor - Electrical energy generated/demanded (kWh),Motor - Fuel consumed (L),Motor - Fuel consumption (L/h),Motor - Fuel consumption (g/kWh)
0,126.0,0.612544,17.968774,17.968774,12872.727121,35.539057,35.375980,582.031226,1.035082,1.004084,...,4.568148,69.804956,59.320897,59.320897,14.004599,34.841631,0.735,0.219162,6.261776,241.52564
1,252.0,1.257686,18.883745,18.883745,11647.839023,35.011888,35.514630,581.116255,1.041153,1.008385,...,4.566155,69.922204,60.502912,60.502912,14.004599,34.841631,1.470,0.438324,6.261776,241.52564
2,378.0,1.934273,19.765964,19.765964,10593.097714,34.557963,35.645009,580.234036,1.047400,1.012895,...,4.564204,70.056667,61.635408,61.635408,14.004599,34.841631,2.205,0.657486,6.261776,241.52564
3,504.0,2.641169,20.616029,20.616029,9685.601913,34.167420,35.767728,579.383971,1.053817,1.017608,...,4.562286,70.211266,62.721117,62.721117,14.004599,34.841631,2.940,0.876649,6.261776,241.52564
4,630.0,3.377262,21.434766,21.434766,8905.500647,33.831712,35.883366,578.565234,1.060397,1.022515,...,4.560393,70.389225,63.762590,63.762590,14.004599,34.841631,3.675,1.095811,6.261776,241.52564


##### Separando apenas o tempo final do output gerado no report.csv

In [235]:
report_filtrado.tail()

,Time (s),Desal - Total water produced (kg),Desal - Distilled water production rate (kg/h),FeedTank - Brackish water inlet flow rate (kg/h),ColdHex - Heat transfer rate (W),ColdHex - Cold side outlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Hot feedwater outlet mass flow rate (kg/h),Desal - Hot feedwater outlet salinity (wt%),HotHex - Cold side salinity (wt%),...,Motor - Nominal Mass flow rate (kg/s),Motor - Mixture temperature engine (°C),Motor - Mixture temperature radiator (°C),Motor - Oulet temperature radiator (°C),Motor - Heat coolant (kW),Motor - System efficiency,Motor - Electrical energy generated/demanded (kWh),Motor - Fuel consumed (L),Motor - Fuel consumption (L/h),Motor - Fuel consumption (g/kWh)
195,24696.0,215.817168,30.779320,30.779320,4428.568691,31.905289,37.178568,569.220680,2.570653,2.438781,...,4.530821,76.689920,75.918763,75.918763,14.004599,34.841631,144.060,42.955782,6.261776,241.52564
196,24822.0,216.891385,30.604305,30.604305,4423.043975,31.902912,37.159431,569.395695,2.577409,2.445943,...,4.531415,76.526410,75.722049,75.722049,14.004599,34.841631,144.795,43.174944,6.261776,241.52564
197,24948.0,217.959310,30.419895,30.419895,4416.873750,31.900258,37.139124,569.580105,2.584074,2.453062,...,4.532040,76.354731,75.513988,75.513988,14.004599,34.841631,145.530,43.394107,6.261776,241.52564
198,25074.0,219.020621,30.226552,30.226552,4410.062446,31.897327,37.117683,569.773448,2.590648,2.460137,...,4.532695,76.175515,75.295020,75.295020,14.004599,34.841631,146.265,43.613269,6.261776,241.52564
199,25200.0,220.075014,30.024721,30.024721,4402.615901,31.894123,37.095142,569.975279,2.597130,2.467167,...,4.533377,75.989396,75.065559,75.065559,14.004599,34.841631,147.000,43.832431,6.261776,241.52564


In [236]:
report_filtrado = report_filtrado.iloc[[report_filtrado['SolarField - Solar irradiance (W/m²)'].idxmax()]].reset_index(drop=True)
report_filtrado.head()

,Time (s),Desal - Total water produced (kg),Desal - Distilled water production rate (kg/h),FeedTank - Brackish water inlet flow rate (kg/h),ColdHex - Heat transfer rate (W),ColdHex - Cold side outlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Hot feedwater outlet mass flow rate (kg/h),Desal - Hot feedwater outlet salinity (wt%),HotHex - Cold side salinity (wt%),...,Motor - Nominal Mass flow rate (kg/s),Motor - Mixture temperature engine (°C),Motor - Mixture temperature radiator (°C),Motor - Oulet temperature radiator (°C),Motor - Heat coolant (kW),Motor - System efficiency,Motor - Electrical energy generated/demanded (kWh),Motor - Fuel consumed (L),Motor - Fuel consumption (L/h),Motor - Fuel consumption (g/kWh)
0,11088.0,94.532429,32.44429,32.44429,4479.516305,31.92721,37.285106,567.55571,1.723407,1.630216,...,4.527584,77.235808,76.566238,58.833258,14.004599,34.841631,64.68,19.28627,6.261776,241.52564


##### Lendo o arquivo do Inkscape e definindo uma função de substituição de valores

In [237]:
# Caminho para o arquivo SVG
arquivo_svg = 'Diag_geral_2025_04_10.svg'

# Parse o arquivo SVG
tree = ET.parse(arquivo_svg)
root = tree.getroot()

# Definir namespaces
namespaces = {'svg': 'http://www.w3.org/2000/svg', 'inkscape': 'http://www.inkscape.org/namespaces/inkscape'}

def substituir_texto_por_label(label, novo_texto):
    # Encontre todos os elementos do tipo 'text'
    textos = root.findall('.//svg:text', namespaces)
    
    # Para cada elemento 'text' encontrado
    for texto in textos:
        # Verifique se o elemento 'text' possui o atributo 'inkscape:label'
        if texto.attrib.get('{http://www.inkscape.org/namespaces/inkscape}label') == label:
            # Itere sobre os elementos 'tspan' dentro do elemento 'text'
            for tspan in texto.findall('.//svg:tspan', namespaces):
                # Substitua o texto do 'tspan' pelo novo texto
                tspan.text = novo_texto

#### Definindo Função para Cálculo da Salinidade em g/l

Atualmente o código gera no report a salinidade em porcentagem de peso wt% (weight percent), porém a salinidade depende da densidade que é inversamente proporcional a temperatura, por isso é necessário o uso da função a seguir:

In [238]:
def salt_water_density(T, s):
    a0 = 9.999 * 10**2
    a1 = 2.034 * 10**-2
    a2 = -6.162 * 10**-3
    a3 = 2.261 * 10**-5
    a4 = -4.657 * 10**-8
    b0 = 8.020 * 10**2
    b1 = -2.001
    b2 = 1.677 * 10**-2
    b3 = -3.060 * 10**-5
    b4 = -1.613 * 10**-5

    result = (a0 + a1 * T + a2 * T**2 + a3 * T**3 + a4 * T**4 + 
              b0 * s + b1 * s * T + b2 * s * T**2 + 
              b3 * s * T**3 + b4 * s**2 * T**2)

    return result

#### Definindo cada a temperatura para cada Salinidade

In [239]:
# De cima para baixo no desenho e da esquerda para a direita, temos

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### HotHex - Cold side salinity (wt%) -> está presente na primeira e na segunda instância de salinidade, os valores são iguais em wt% mas distintos em g/L


# A temperatura da primeira instância (mais a esquerda) é a variável HotHex - Cold side outlet temperature (°C)

# No desenho, o nome dessa instância da salinidade foi alterado para HotHex - Cold side salinity 1 (wt%)

T_css1 = float(report_filtrado['HotHex - Cold side outlet temperature (°C)'].iloc[0])
S_css1 = float(report_filtrado['HotHex - Cold side salinity (wt%)'].iloc[0]/100)
css1_g_l = "s " + str(round(S_css1 * salt_water_density(T_css1, S_css1), 2)) + " g/l"

# A temperatura da segunda instância (mais a direita) é a variável HotHex - Cold side inlet temperature (°C)
# No desenho, o nome dessa instância da salinidade foi alterado para HotHex - Cold side salinity 2 (wt%)

T_css2 = float(report_filtrado['HotHex - Cold side inlet temperature (°C)'].iloc[0])
S_css2 = float(report_filtrado['HotHex - Cold side salinity (wt%)'].iloc[0]/100)
css2_g_l = "s " + str(round(S_css2 * salt_water_density(T_css2, S_css2), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Desal - Hot feedwater outlet salinity (wt%) -> terceira instância de salinidade

# A temperatura da terceira instância está na variável -> Desal - Hot feedwater outlet temperature (°C)

T_hfos = float(report_filtrado['Desal - Hot feedwater outlet temperature (°C)'].iloc[0])
S_hfos = float(report_filtrado['Desal - Hot feedwater outlet salinity (wt%)'].iloc[0]/100)
hfos_g_l = "s " + str(round(S_hfos * salt_water_density(T_hfos, S_hfos), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### ColdHex - Hot side salinity (wt%) -> está presente na quarta e quinta instância de salinidade

# A temperatura da quarta instância (mais para cima) é a variável ColdHex - Hot side outlet temperature (°C)
# No desenho, o nome dessa instância da salinidade foi alterado para ColdHex - Hot side salinity 1 (wt%)

T_hss1 = float(report_filtrado['ColdHex - Hot side outlet temperature (°C)'].iloc[0])
S_hss1 = float(report_filtrado['ColdHex - Hot side salinity (wt%)'].iloc[0]/100)
hss1_g_l = "s " + str(round(S_hss1 * salt_water_density(T_hss1, S_hss1), 2)) + " g/l"

# A temperatura da quinta instância (mais para baixo) é a variável ColdHex - Hot side inlet temperature (°C)
# No desenho, o nome dessa instância da salinidade foi alterado para ColdHex - Hot side salinity 2 (wt%)

T_hss2 = float(report_filtrado['ColdHex - Hot side inlet temperature (°C)'].iloc[0])
S_hss2 = float(report_filtrado['ColdHex - Hot side salinity (wt%)'].iloc[0]/100)
hss2_g_l = "s " + str(round(S_hss2 * salt_water_density(T_hss2, S_hss2), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Brackish water salinity at the entrance of the regulating tank = -> está presente na sexta instância de salinidade

# A temperatura da sexta instância está na variável -> Brackish water temperature at the entrance of the regulating tank =

T_bws = float(entry_data_filtrado[entry_data_filtrado['Entry data:'] == 'Brackish water temperature at the entrance of the regulating tank =']['Unnamed: 1'].iloc[0])
S_bws = float(entry_data_filtrado[entry_data_filtrado['Entry data:'] == 'Brackish water salinity at the entrance of the regulating tank =']['Unnamed: 1'].iloc[0])/100
bws_g_l = "s " + str(round(S_bws * salt_water_density(T_bws, S_bws), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### FeedTank - Water salinity (wt%) -> está presente na sétima instância de salinidade

# A temperatura da sétima instância está na variável -> FeedTank - Water temperature (°C)

T_fts = float(report_filtrado['FeedTank - Water temperature (°C)'].iloc[0])
S_fts = float(report_filtrado['FeedTank - Water salinity (wt%)'].iloc[0]/100)
fts_g_l = "s " + str(round(S_fts * salt_water_density(T_fts, S_fts), 2)) + " g/l"

#### Passando o Desal - Total water produced (kg) para litro -> vamos dividir o valor em kg por 0.998

In [240]:
total_water_produced_kg = report_filtrado['Desal - Total water produced (kg)'].iloc[0]
total_water_produced_l = str(round(total_water_produced_kg / 0.998, 2)) + " L"
print(total_water_produced_l)

94.72 L


#### Calculando o valor de (Desal - Film wall temperature (°C)+Desal - Gap film boundary temperature (°C))/2

In [241]:
film_wall_temp = report_filtrado['Desal - Film wall temperature (°C)'].iloc[0]
gap_film_boundary_temp = report_filtrado['Desal - Gap film boundary temperature (°C)'].iloc[0]
average_temp = str(round((film_wall_temp + gap_film_boundary_temp) / 2, 2)) + " °C"

#### Calculando o valor de Motor - Radiator in mass flow rate (kg/s) + Motor - Radiator bypass mass flow rate (kg/s)

In [242]:
motor_flow_rate_combined = str(round(report_filtrado['Motor - Radiator in mass flow rate (kg/s)'].iloc[0] + report_filtrado['Motor - Radiator bypass mass flow rate (kg/s)'].iloc[0], 2)) + " kg/s"

##### Substituindo os Valores

In [243]:
for coluna, valor in report_filtrado.items():
    valor = valor.iloc[0]
    grandeza = ""
    if '(' in coluna and ')' in coluna:
        unidade = coluna[coluna.rfind('(') + 1:coluna.rfind(')')]
        unidade = unidade.strip()
    elif coluna == "SolarField - Efficiency" or coluna == "PVT - Thermal Efficiency" or coluna == "ETC - Efficiency" or coluna == "PVT - Electrical Efficiency" or coluna == "Motor - System efficiency":
        unidade = "%"
    else:
        unidade = ""
    if coluna == "PVT - Heat transfer rate (W)" or coluna == "SolarField - Heat transfer rate (W)" or coluna == "ETC - Heat transfer rate (W)" or coluna == "PVT - Electrical power output (W)":
        unidade = "kW"
        valor = "{:.2f}".format(float(valor/1000))
    if unidade == '°C':
        grandeza = "T"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'kg/s':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'Kg/h' or unidade == 'kg/h':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'g/L':
        grandeza = "S"
        valor = "{:.2f}".format(float(valor))
    valor = "{:.2f}".format(float(valor))
    valor_novo = grandeza + " " + str(valor) + " " + unidade
    print(valor_novo)
    substituir_texto_por_label(coluna, valor_novo)

 11088.00 s
 94.53 kg
ṁ 32.44 kg/h
ṁ 32.44 kg/h
 4479.52 W
T 31.93 °C
T 37.29 °C
ṁ 567.56 kg/h
 1.72 wt%
 1.63 wt%
T 30.06 °C
 1.63 wt%
 4951.17 W
T 77.33 °C
T 77.77 °C
T 77.75 °C
T 36.62 °C
ṁ 600.00 kg/h
T 70.52 °C
 952.20 W/m²
 34.12 %
T 76.14 °C
T 79.20 °C
T 79.09 °C
T 76.31 °C
 1.63 wt%
T 36.62 °C
T 51.56 °C
T 53.75 °C
T 76.31 °C
T 69.52 °C
ṁ 0.34 kg/s
 -8.63 kW
 4.06 kW
 23.69 %
 11.04 kWh
ṁ 0.34 kg/s
 55.09 %
T 76.31 °C
T 79.07 °C
ṁ 0.23 kg/s
T 76.31 °C
 40.03 %
 0.49 kW
 13.73 %
 1.35 kWh
T 78.00 °C
ṁ 0.23 kg/s
 1.43 kW
 2.34 kW
 3.90 kW
T 76.31 °C
ṁ 0.50 kg/s
ṁ 76.31 kg/s
T 77.77 °C
ṁ 3.00 kg/s
ṁ 2.50 kg/s
T 78.06 °C
ṁ 0.10 kg/s
ṁ 2.40 kg/s
ṁ 2.03 kg/s
ṁ 4.53 kg/s
T 77.24 °C
T 76.57 °C
T 58.83 °C
 14.00 kW
 34.84 %
 64.68 kWh
 19.29 L
 6.26 L/h
 241.53 g/kWh


In [244]:
for linha in entry_data_filtrado.itertuples():
    grandeza = ""
    label = linha[1].rstrip(' ')
    valor = linha[2]
    valor = "{:.2f}".format(float(valor))
    unidade = linha[3].strip()
    if unidade == '°C':
        grandeza = "T"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'kg/s':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'kg/h' or unidade == 'Kg/h':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'g/L':
        grandeza = "S"
        valor = "{:.2f}".format(float(valor))

    valor_novo = grandeza + " " + str(valor) + ' ' + unidade
    #print(valor_novo)
    substituir_texto_por_label(label, str(valor_novo))

In [245]:
substituir_texto_por_label("HotHex - Cold side salinity 1 (wt%)", css1_g_l)
substituir_texto_por_label("HotHex - Cold side salinity 2 (wt%)", css2_g_l)
substituir_texto_por_label("Desal - Hot feedwater outlet salinity (wt%)", hfos_g_l)
substituir_texto_por_label("ColdHex - Hot side salinity 1 (wt%)", hss1_g_l)
substituir_texto_por_label("ColdHex - Hot side salinity 2 (wt%)", hss2_g_l)
substituir_texto_por_label("Brackish water salinity at the entrance of the regulating tank =", bws_g_l)
substituir_texto_por_label("FeedTank - Water salinity (wt%)", fts_g_l)
substituir_texto_por_label("Motor - Radiator in mass flow rate (kg/s) + Motor - Radiator bypass mass flow rate (kg/s)", motor_flow_rate_combined)

# correção do valor de total de água produzida
substituir_texto_por_label("Desal - Total water produced (kg)", total_water_produced_l)

# correção do valor de temperatura média
substituir_texto_por_label("(Desal - Film wall temperature (°C)+Desal - Gap film boundary temperature (°C))/2", average_temp)

##### Salvando o arquivo

In [246]:
# Salvar as alterações no arquivo SVG
tree.write('Diag_geral_modificado.svg')